In [ ]:
%matplotlib
import numpy as np
import matplotlib.pyplot as plt
from ipyparallel import Client
clnts = Client(profile='mpi')
clnts[:].clear()
clnts[:].activate()
%px from mpi4py.MPI import COMM_WORLD as comm
mpiindx = range(len(clnts))
for i in xrange(len(clnts)): mpiindx[clnts[i]['comm.rank']] = i
view = clnts[mpiindx]

mpistackR = lambda x:np.vstack((view[x]))
mpistackD = lambda x: np.sum((view[x]),axis=0)
mpistackP = lambda x: np.concatenate((view[x]),axis=-1)
def mpistackRover(x):
    loc_arrays = []
    for i in xrange(len(view[x])-1):
        dat = view[x][i].copy()
        dat[:,-1] += view[x][i+1][:,0]
        dat = dat[:,1:]
        loc_arrays.append(dat)
    return np.hstack((loc_arrays))

In [ ]:
%%px --block --verbose

import numpy as np
from sys import path
path.append('./src/')
from species import Specie
from solvers import Solver
from chimera_main import ChimeraRun

BoxGrid = (0.,20.,10.,0.1,0.1)
dt=0.5
fu = np.vectorize(lambda x,y,z:np.float(((x-10)**2+y**2+z**2)**0.5<4.))

specie_in = { 'RandCell':30, 'Charge':-1, 'Density':1., 'Mass':1., 'Grid':BoxGrid, 
'TimeStep':dt,'MomentaSpreads':(0.1,0.1,0.1),'Features':('Rsliced')}
solver_in = {'MaxAzimuthMode':0,'Grid':BoxGrid,'TimeStep':dt, 'Features':('Rsliced','SpaceCharge')}
solver = Solver(comm,solver_in) 

#MovingFrame = {'TimeStep':1.,'Steps':1,'AbsorbLayer':3}

specie = Specie(comm,specie_in)
specie.add_particles(specie.gen_parts((5.,15.,0.,4),fu))

chimera_in = {'Solvers':(solver,),'Particles':(specie,),'MovingFrames':()}
Chimera = ChimeraRun(comm,chimera_in)


In [ ]:
%%px --block --verbose
Chimera.make_halfstep()

In [ ]:
%px dens = specie.get_dens_on_grid()
plt.clf()
plt.imshow( mpistackRover("np.abs(dens[:,:,0])").T,aspect='auto',origin='lower',cmap='spectral',extent=(0,20,0,10))
plt.colorbar()

In [ ]:
%%px --block --verbose
for i in range(10,15):
    Chimera.make_step(i)

In [ ]:
plt.clf()
plt.plot(mpistackP('specie.particles[0]'),mpistackP('specie.particles[1]'),'.')